In [1]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab or
# Jupyter Notebook in an environment where the library is not pre-installed.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        !pip install 'easydiffraction[visualization]'


# Pair Distribution Function: Si, NPD

This example demonstrates a pair distribution function (PDF) analysis of Si,
based on data collected from a time-of-flight neutron powder diffraction
experiment at NOMAD at SNS.

## Import Library

In [2]:
import easydiffraction as ed

## Create Project

In [3]:
project = ed.Project()

## Set Plotting Engine

In [4]:
project.plotter.engine = 'plotly'
project.plotter.x_max = 40


Current plotter changed to
plotly


## Add Sample Model

In [5]:
project.sample_models.add(name='si')

In [6]:
sample_model = project.sample_models['si']
sample_model.space_group.name_h_m.value = 'F d -3 m'
sample_model.space_group.it_coordinate_system_code = '1'
sample_model.cell.length_a = 5.43146
sample_model.atom_sites.add(label='Si', type_symbol='Si', fract_x=0, fract_y=0, fract_z=0, wyckoff_letter='a', b_iso=0.5)

## Add Experiment

In [7]:
ed.download_from_repository('NOM_9999_Si_640g_PAC_50_ff_ftfrgr_up-to-50.gr', destination='data')


⚠️ Warning
File 'data/NOM_9999_Si_640g_PAC_50_ff_ftfrgr_up-to-50.gr' already exists and will not be overwritten.


In [8]:
project.experiments.add(
    name='nomad',
    sample_form='powder',
    beam_mode='time-of-flight',
    radiation_probe='neutron',
    scattering_type='total',
    data_path='data/NOM_9999_Si_640g_PAC_50_ff_ftfrgr_up-to-50.gr',
)


Loading measured data from ASCII file
/Users/runner/work/diffraction-lib/diffraction-lib/tutorials/data/NOM_9999_Si_640g_PAC_50_ff_ftfrgr_up-to-50.gr

Data loaded successfully
Experiment 🔬 'nomad'. Number of data points: 5033


In [9]:
experiment = project.experiments['nomad']
experiment.linked_phases.add(id='si', scale=1.0)
experiment.peak.damp_q = 0.02
experiment.peak.broad_q = 0.03
experiment.peak.cutoff_q = 35.0
experiment.peak.sharp_delta_1 = 0.0
experiment.peak.sharp_delta_2 = 4.0
experiment.peak.damp_particle_diameter = 0

## Select Fitting Parameters

In [10]:
project.sample_models['si'].cell.length_a.free = True
project.sample_models['si'].atom_sites['Si'].b_iso.free = True
experiment.linked_phases['si'].scale.free = True

In [11]:
experiment.peak.damp_q.free = True
experiment.peak.broad_q.free = True
experiment.peak.sharp_delta_1.free = True
experiment.peak.sharp_delta_2.free = True

## Run Fitting

In [12]:
project.analysis.current_calculator = 'pdffit'
project.analysis.fit()


Current calculator changed to
pdffit

Using experiment 🔬 'nomad' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,3102.63,
11,2177.55,29.8% ↓
19,345.28,84.1% ↓
27,180.23,47.8% ↓
35,170.60,5.3% ↓
68,170.54,


🏆 Best goodness-of-fit (reduced χ²) is 170.54 at iteration 67
✅ Fitting complete.

Fit results
✅ Success: True
⏱️ Fitting time: 9.72 seconds
📏 Goodness-of-fit (reduced χ²): 170.54
📏 R-factor (Rf): 8.40%
📏 R-factor squared (Rf²): 8.30%
📏 Weighted R-factor (wR): 8.30%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,si,atom_sites,Si,b_iso,0.5000,0.7170,0.0050,Å²,43.40 % ↑
2,si,cell,,length_a,5.4315,5.4306,0.0000,Å,0.02 % ↓
3,nomad,linked_phases,si,scale,1.0000,1.2728,0.0014,,27.28 % ↑
4,nomad,peak,,broad_q,0.0300,0.0183,0.0003,Å⁻²,39.10 % ↓
5,nomad,peak,,damp_q,0.0200,0.0251,0.0001,Å⁻¹,25.66 % ↑
6,nomad,peak,,sharp_delta_1,0.0000,2.5400,0.0529,Å,N/A
7,nomad,peak,,sharp_delta_2,4.0000,-1.7525,0.1244,Å²,143.81 % ↓


## Plot Measured vs Calculated

In [13]:
project.plot_meas_vs_calc(expt_name='nomad', show_residual=False)